## 任务8.4　使用灰色预测和`SVR`构建财政收入预测模型

<center>
    <br>
    代码8-3　财政收入灰色预测
</center>

In [1]:
# 导入所需包
import numpy
import pandas

# 自定义灰度预测函数
def gray_prediction(x_0):
    x_1 = x_0.cumsum()
    z_1 = (x_1[:len(x_1) - 1] + x_1[1:]) / 2.0
    z_1 = z_1.reshape((len(z_1), 1))
    B = numpy.append(-z_1, numpy.ones_like(z_1), axis=1)
    Y_n = x_0[1:].reshape((len(x_0) - 1, 1))
    [[a], [b]] = numpy.dot(numpy.dot(numpy.linalg.inv(numpy.dot(B.T, B)), B.T), Y_n)
    f = lambda k: (x_0[0] - b / a) * numpy.exp(-a * (k - 1)) - (x_0[0] - b / a) * numpy.exp(-a * (k - 2))
    delta = numpy.abs(x_0 - numpy.array([f(i) for i in range(1, len(x_0) + 1)]))
    C = delta.std() / x_0.std()
    P = 1.0 * (numpy.abs(delta - delta.mean()) < 0.6745 * x_0.std()).sum() / len(x_0)
    return (f, a, b, x_0[0], C, P)

new_reg_data = pandas.read_csv('stash/new_reg_data.csv')
data = pandas.read_csv('data/data_task.csv')
new_reg_data.index = range(1994, 2014)
new_reg_data.loc[2014] = None
new_reg_data.loc[2015] = None
for i in ['x1', 'x3', 'x4', 'x5', 'x6', 'x7', 'x8', 'x13']:
    f = gray_prediction(new_reg_data.loc[range(1994, 2014), i].values)[0]
    new_reg_data.loc[2014, i] = f(len(new_reg_data) - 1)
    new_reg_data.loc[2015, i] = f(len(new_reg_data))
    new_reg_data[i] = new_reg_data[i].round(2)
y = list(data['y'].values)
y.extend([numpy.NaN, numpy.NaN])
new_reg_data['y'] = y
new_reg_data.to_csv('stash/new_reg_data_gray.csv')
print('预测结果为：', new_reg_data.loc[2014:2015, :], sep='\n')

预测结果为：
      Unnamed: 0          x1       x3        x4        x5          x6  \
2014         NaN  8142148.24  7042.31  43611.84  35046.63  8505522.58   
2015         NaN  8460489.28  8166.92  47792.22  38384.22  8627139.31   

           x7        x8       x13   y  
2014  4600.40  18686.28  44506.47 NaN  
2015  5214.78  21474.47  49945.88 NaN  


<center>
    <br>
    代码8-4　财政收入支持向量回归预测模型
</center>

In [2]:
from sklearn.svm import LinearSVR
import matplotlib.pyplot as plt
from sklearn.metrics import explained_variance_score, mean_absolute_error,mean_squared_error
from sklearn.metrics import median_absolute_error,r2_score
data = pandas.read_csv('stash/new_reg_data_gray.csv')
feature = ['x1', 'x3', 'x4', 'x5', 'x6', 'x7', 'x8', 'x13'] 
data_train = data.loc[range(1994, 2014)].copy()
data_mean = data_train.mean()
data_std = data_train.std()
data_train = (data_train - data_mean) / data_std
x_train = data_train[feature].values
y_train = data_train['y'].values
linearsvr = LinearSVR()
linearsvr.fit(x_train,y_train)
x = ((data[feature] - data_mean[feature]) / data_std[feature]).values
data['y_pred'] = linearsvr.predict(x) * data_std['y'] + data_mean['y'] 
data.to_csv('stash/new_reg_data_gray_revenue.csv')
print('真实值与预测值分别为：', data[['y','y_pred']])

KeyError: "None of [Int64Index([1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004,\n            2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013],\n           dtype='int64')] are in the [index]"

In [3]:
print('预测图为：', data[['y', 'y_pred']].plot(subplots=True, style=['b-o', 'r-*']))

KeyError: "['y_pred'] not in index"